In [27]:
import pandas as pd
import numpy as np
import matplotlib as plt
from kalman import AircraftKF

In [6]:
df = pd.read_parquet("snapshots/2025/08/17/22/adsb_mr_2025-08-17T22-40-00Z_10min.parquet")

In [7]:
df.head()

,t_rx,hex,flight,lat,lon,altitude,speed,track,vert_rate,rssi,seen,seen_pos,messages,category
0,1.755471e+09,a07c05,N130JM,34.117676,-118.436092,3700,87,119,512,None,1,None,38,None
1,1.755471e+09,a979b7,N71AC,34.034946,-118.470955,3600,136,140,-256,None,1,None,315,None
2,1.755471e+09,aa39f8,None,0.000000,0.000000,400,0,0,0,None,28,None,52,None
3,1.755471e+09,a687cc,N52NL,34.017256,-118.434448,800,112,34,-320,None,10,None,964,None
4,1.755471e+09,a99565,N7165G,34.007446,-118.487773,1800,84,22,576,None,1,None,1169,None


In [9]:
df = df[df["flight"].str.startswith("N", na=False)]

In [17]:
counts = df.groupby("flight").size().reset_index(name="count")
counts

,flight,count
0,N130JM,169
1,N1998R,263
2,N231PN,43
3,N4388A,265
4,N52NL,409
5,N661CV,51
6,N7165G,409
7,N71AC,397
8,N844AT,131
9,N8451N,106


In [21]:
df[df["flight"] == "N52NL"]

,t_rx,hex,flight,lat,lon,altitude,speed,track,vert_rate,rssi,seen,seen_pos,messages,category
3,1.755471e+09,a687cc,N52NL,34.017256,-118.434448,800,112,34,-320,None,10,None,964,None
14,1.755471e+09,a687cc,N52NL,34.017256,-118.434448,800,112,34,-320,None,11,None,964,None
25,1.755471e+09,a687cc,N52NL,34.017256,-118.434448,800,112,34,-320,None,12,None,964,None
36,1.755471e+09,a687cc,N52NL,34.017256,-118.434448,800,112,34,-320,None,13,None,964,None
47,1.755471e+09,a687cc,N52NL,34.017256,-118.434448,800,112,34,-320,None,14,None,964,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3998,1.755471e+09,a687cc,N52NL,34.012184,-118.453651,325,5,241,-320,None,227,None,975,None
4006,1.755471e+09,a687cc,N52NL,34.012184,-118.453651,325,5,241,-320,None,228,None,975,None
4014,1.755471e+09,a687cc,N52NL,34.012184,-118.453651,325,5,241,-320,None,229,None,975,None
4022,1.755471e+09,a687cc,N52NL,34.012184,-118.453651,325,5,241,-320,None,230,None,975,None


In [22]:
KEYS = ["flight","hex","lat","lon","altitude","speed","track","vert_rate"]

# keep the last occurrence (or use keep="first")
df_dedup = (df
    .drop_duplicates(subset=KEYS, keep="last")
    .sort_values(["flight","hex","t_rx"], kind="mergesort")
    .reset_index(drop=True)
)

In [24]:
df_dedup["flight"].value_counts()

flight
N7165G    339
N1998R    220
N130JM    151
N93GS     141
N4388A    125
N71AC      88
N661CV     50
N231PN     42
N52NL       7
N844AT      1
N8451N      1
Name: count, dtype: int64

In [30]:
best_track_flights = ["N130JM", "N7165G", "N231PN"]

In [28]:
kdf = df.sort_values("t_rx")

In [32]:
def key(row):
    f = (row.get("flight") or "").strip()
    return f if f else row.get("hex")

filters = {}
pred_rows = []  # store predictions alongside measurements

for _, r in kdf.iterrows():
    k = key(r)
    if not k: 
        continue
    if k not in filters:
        filters[k] = AircraftKF()

    kf = filters[k]
    kf.update(
        t_meas=float(r["t_rx"]),
        altitude_ft=r.get("altitude"),
        speed_kt=r.get("speed"),
        track_deg=r.get("track"),
        vert_rate_fpm=r.get("vert_rate"),
    )
    # Predict 15 seconds ahead (tweak to taste)
    alt_ft, spd_kt, track_deg = kf.predict_next(dt_ahead=15.0)
    pred_rows.append({
        "name": k,
        "t_rx": r["t_rx"],
        "alt_pred_ft_15s": alt_ft,
        "speed_pred_kt_15s": spd_kt,
        "track_pred_deg_15s": track_deg
    })

pred_df = pd.DataFrame(pred_rows)

In [33]:
pred_df

,name,t_rx,alt_pred_ft_15s,speed_pred_kt_15s,track_pred_deg_15s
0,N130JM,1.755471e+09,3828.000000,87.000000,119.000000
1,N71AC,1.755471e+09,3536.000000,136.000000,140.000000
2,N52NL,1.755471e+09,720.000000,112.000000,34.000000
3,N7165G,1.755471e+09,1944.000000,84.000000,22.000000
4,N93GS,1.755471e+09,3303.000000,114.000000,56.000000
...,...,...,...,...,...
2647,N7165G,1.755471e+09,3425.536586,98.792102,57.543264
2648,N1998R,1.755471e+09,4464.896811,109.899895,280.123001
2649,N231PN,1.755471e+09,1186.149026,125.953802,52.431235
2650,N661CV,1.755471e+09,1524.823819,117.626805,267.755494


In [31]:
filters

{'N130JM': <kalman.AircraftKF at 0x12350c110>,
 'N71AC': <kalman.AircraftKF at 0x123091150>,
 'N52NL': <kalman.AircraftKF at 0x1235c5cd0>,
 'N7165G': <kalman.AircraftKF at 0x1235e2850>,
 'N93GS': <kalman.AircraftKF at 0x1235c6050>,
 'N844AT': <kalman.AircraftKF at 0x1235e3550>,
 'N8451N': <kalman.AircraftKF at 0x12361b810>,
 'N4388A': <kalman.AircraftKF at 0x1227f2d10>,
 'N1998R': <kalman.AircraftKF at 0x120d3d4d0>,
 'N661CV': <kalman.AircraftKF at 0x12361a310>,
 'N231PN': <kalman.AircraftKF at 0x1235fa2d0>}